In [1]:
import pandas as pd

df = pd.read_csv('../dados/jurisprudencia_selecionada_excertos.CSV', sep = ';')
df.head()

,COD,NUM_ENUNCIADO,COD_AREA,DESCR_AREA,COD_TEMA,DESCR_TEMA,COD_SUBTEMA,DESCR_SUBTEMA,COD_DOC_TRAMITAVEL_ENUNCIADO,TEXTO_ENUNCIADO,ACORDAO,TIPO_PROCESSO
0,1400,1236,50,Responsabilidade,488,Solidariedade,261,Benefício previdenciário,54995437,Não comprovada a participação do beneficiário ...,Acórdão 297/2016 - PL,Tomada de Contas Especial
1,1700,1534,46,Finanças Públicas,981,Exportação,983,Petróleo,55025587,A operação ficta de exportação de plataformas ...,Acórdão 366/2016 - PL,Solicitação do Congresso Nacional
2,5700,5314,50,Responsabilidade,203,Multa,1021,Dosimetria,55455370,"No âmbito do TCU, a dosimetria da pena tem com...",Acórdão 944/2016 - PL,Acompanhamento
3,284,40,45,Direito Processual,162,Princípio da independência das instâncias,481,Decisão judicial,54773746,O princípio da independência das instâncias pe...,Acórdão 30/2016 - PL,Tomada de Contas Especial
4,298,54,49,Pessoal,141,Sistema S,142,Nepotismo,54773402,É vedado aos dirigentes das entidades do Siste...,Acórdão 55/2016 - PL,Representação


In [2]:
df.shape

(13312, 12)

In [3]:
areas = df.groupby(['DESCR_AREA']).groups.keys()
areas

dict_keys(['Competência do TCU', 'Contrato Administrativo', 'Convênio', 'Desestatização', 'Direito Processual', 'Finanças Públicas', 'Gestão Administrativa', 'Licitação', 'Pessoal', 'Responsabilidade'])

In [4]:
from sklearn.preprocessing import LabelBinarizer

lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [5]:
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13312, 10)

In [6]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 20000
limite_texto = 200
dim_vetor = 100

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['TEXTO_EXCERTO'])
vocabulario = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(df['TEXTO_EXCERTO'])

Using TensorFlow backend.


In [7]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13312, 200)


In [8]:
alternativas = [None, 'sem pré-treino', 'NILC fixo', 'NILC variável', 'Acordãos fixo', 'Acordãos variável']

In [9]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

print('Montando embeddings NILC:')
model_nilc = KeyedVectors.load_word2vec_format('../externos/model.txt')
embedding_matrix_nilc = np.zeros((vocabulario, dim_vetor))
ok = 0
for word, i in tokenizer.word_index.items():
    if word in model_nilc:
        embedding_matrix_nilc[i] = model_nilc[word]
        ok += 1
print('\tVocabulario:', i, ' - encontrados no modelo:', ok, '=', ok * 100. / i)

print('Montando embeddings Acordaos:')
model_acordaos = Word2Vec.load('../vocabularios/modelo-acordaos.w2v')
embedding_matrix_acordaos = np.zeros((vocabulario, dim_vetor))
ok = 0
for word, i in tokenizer.word_index.items():
    if word in model_acordaos.wv:
        embedding_matrix_acordaos[i] = model_acordaos.wv[word]
        ok += 1
print('\tVocabulario:', i, ' - encontrados no modelo:', ok, '=', ok * 100. / i)

embeddings = [None, None, embedding_matrix_nilc, embedding_matrix_nilc, embedding_matrix_acordaos, embedding_matrix_acordaos]

Montando embeddings NILC:


/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


	Vocabulario: 15387  - encontrados no modelo: 13758 = 89.41314096315071
Montando embeddings Acordaos:
	Vocabulario: 15387  - encontrados no modelo: 14572 = 94.70332098524729


In [10]:
colunas_scores = list(lbArea.classes_)
colunas_scores.extend(['accuracy', 'macro avg', 'weighted avg'])

In [12]:
def gera_embedding(alt, model, vocabulario, dim_vetor, limite_texto):
        if alt == 1:
            model.add(Embedding(vocabulario, dim_vetor, input_length=limite_texto, trainable=True))
        elif alt == 2:
            model.add(Embedding(vocabulario, dim_vetor, input_length=limite_texto, trainable=False, weights=[embedding_matrix_nilc]))
        elif alt == 3:
            model.add(Embedding(vocabulario, dim_vetor, input_length=limite_texto, trainable=True,  weights=[embedding_matrix_nilc]))
        elif alt == 4:
            model.add(Embedding(vocabulario, dim_vetor, input_length=limite_texto, trainable=False, weights=[embedding_matrix_acordaos]))
        elif alt == 5:
            model.add(Embedding(vocabulario, dim_vetor, input_length=limite_texto, trainable=True,  weights=[embedding_matrix_acordaos]))

In [14]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint

print('Realizando treinamentos:')
df_medias = pd.DataFrame()
for alt in range(1, 6):
    str_alt = str(alt)
    fold = 0
    df_scores = pd.DataFrame()
    for train_index, val_index in KFold(n_splits=10, random_state=42, shuffle=True).split(df):
        str_fold = 'Fold ' + str(fold)
        print()
        print('Alternativa ' + alternativas[alt], '-', str_fold + ':')

        df_train = df.loc[train_index]
        df_val = df.loc[val_index]

        sequences_train = tokenizer.texts_to_sequences(df_train['TEXTO_EXCERTO'])
        sequences_val = tokenizer.texts_to_sequences(df_val['TEXTO_EXCERTO'])

        x_train = pad_sequences(sequences_train, maxlen=limite_texto)
        x_val = pad_sequences(sequences_val, maxlen=limite_texto)

        y_train = lbArea.transform(df_train['DESCR_AREA'])
        y_val = lbArea.transform(df_val['DESCR_AREA'])

        model = Sequential()
        gera_embedding(alt, model, vocabulario, dim_vetor, limite_texto)
        model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
        model.add(Dense(y.shape[1], activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=['categorical_accuracy'])

        checkpoint_filename = 'weights' + str_alt + '-' + str(fold) + '.hdf5'
        checkpointer = ModelCheckpoint(filepath=checkpoint_filename, monitor='val_categorical_accuracy', verbose=1, save_best_only=True)
        model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_val, y_val), verbose=1, shuffle=False, callbacks=[checkpointer])

        print('Evaluating best model and registering score:')
        model.load_weights(checkpoint_filename)
        y_val_pred = model.predict_classes(x_val, verbose=1)
        y_val_i = [list(x).index(1) for x in y_val]
        report = classification_report(y_val_i, y_val_pred, target_names=lbArea.classes_, output_dict = True)
        for col in colunas_scores:
            if col == 'accuracy':
                f = report[col]
            else:
                f = report[col]['f1-score']
            df_scores.loc[str_fold,col] = f
        fold += 1
    df_medias[alternativas[alt] + ' mean'] = df_scores.mean()
    df_medias[alternativas[alt] + ' std'] = df_scores.std()
df_medias.T

Realizando treinamentos:

Alternativa sem pré-treino - Fold 0:
Train on 11980 samples, validate on 1332 samples
Epoch 1/20
11980/11980 [==============================] - 73s 6ms/step - loss: 1.5604 - categorical_accuracy: 0.4946 - val_loss: 1.3284 - val_categorical_accuracy: 0.5518

Epoch 00001: val_categorical_accuracy improved from -inf to 0.55180, saving model to weights1-0.hdf5
Epoch 2/20
11980/11980 [==============================] - 72s 6ms/step - loss: 0.9839 - categorical_accuracy: 0.6826 - val_loss: 1.0043 - val_categorical_accuracy: 0.6464

Epoch 00002: val_categorical_accuracy improved from 0.55180 to 0.64640, saving model to weights1-0.hdf5
Epoch 3/20
11980/11980 [==============================] - 72s 6ms/step - loss: 0.6800 - categorical_accuracy: 0.7801 - val_loss: 0.7218 - val_categorical_accuracy: 0.7740

Epoch 00003: val_categorical_accuracy improved from 0.64640 to 0.77402, saving model to weights1-0.hdf5
Epoch 4/20
11980/11980 [==============================] - 72s 6

11980/11980 [==============================] - 70s 6ms/step - loss: 0.1311 - categorical_accuracy: 0.9590 - val_loss: 0.4604 - val_categorical_accuracy: 0.8686

Epoch 00011: val_categorical_accuracy did not improve from 0.87012
Epoch 12/20
11980/11980 [==============================] - 71s 6ms/step - loss: 0.1013 - categorical_accuracy: 0.9701 - val_loss: 0.5281 - val_categorical_accuracy: 0.8566

Epoch 00012: val_categorical_accuracy did not improve from 0.87012
Epoch 13/20
11980/11980 [==============================] - 71s 6ms/step - loss: 0.0882 - categorical_accuracy: 0.9745 - val_loss: 0.5227 - val_categorical_accuracy: 0.8551

Epoch 00013: val_categorical_accuracy did not improve from 0.87012
Epoch 14/20
11980/11980 [==============================] - 71s 6ms/step - loss: 0.0723 - categorical_accuracy: 0.9793 - val_loss: 0.5611 - val_categorical_accuracy: 0.8551

Epoch 00014: val_categorical_accuracy did not improve from 0.87012
Epoch 15/20
11980/11980 [===========================


Epoch 00001: val_categorical_accuracy improved from -inf to 0.57100, saving model to weights1-3.hdf5
Epoch 2/20
11981/11981 [==============================] - 70s 6ms/step - loss: 0.9986 - categorical_accuracy: 0.6734 - val_loss: 1.0185 - val_categorical_accuracy: 0.6709

Epoch 00002: val_categorical_accuracy improved from 0.57100 to 0.67092, saving model to weights1-3.hdf5
Epoch 3/20
11981/11981 [==============================] - 71s 6ms/step - loss: 0.6946 - categorical_accuracy: 0.7758 - val_loss: 0.7322 - val_categorical_accuracy: 0.7701

Epoch 00003: val_categorical_accuracy improved from 0.67092 to 0.77010, saving model to weights1-3.hdf5
Epoch 4/20
11981/11981 [==============================] - 70s 6ms/step - loss: 0.5224 - categorical_accuracy: 0.8326 - val_loss: 0.6303 - val_categorical_accuracy: 0.7866

Epoch 00004: val_categorical_accuracy improved from 0.77010 to 0.78663, saving model to weights1-3.hdf5
Epoch 5/20
11981/11981 [==============================] - 71s 6ms/step

11981/11981 [==============================] - 71s 6ms/step - loss: 0.6941 - categorical_accuracy: 0.7695 - val_loss: 0.8101 - val_categorical_accuracy: 0.7340

Epoch 00003: val_categorical_accuracy improved from 0.62810 to 0.73403, saving model to weights1-6.hdf5
Epoch 4/20
11981/11981 [==============================] - 71s 6ms/step - loss: 0.5261 - categorical_accuracy: 0.8317 - val_loss: 0.7103 - val_categorical_accuracy: 0.7648

Epoch 00004: val_categorical_accuracy improved from 0.73403 to 0.76484, saving model to weights1-6.hdf5
Epoch 5/20
11981/11981 [==============================] - 71s 6ms/step - loss: 0.4291 - categorical_accuracy: 0.8629 - val_loss: 0.5741 - val_categorical_accuracy: 0.8212

Epoch 00005: val_categorical_accuracy improved from 0.76484 to 0.82119, saving model to weights1-6.hdf5
Epoch 6/20
11981/11981 [==============================] - 71s 6ms/step - loss: 0.3546 - categorical_accuracy: 0.8902 - val_loss: 0.5862 - val_categorical_accuracy: 0.8249

Epoch 00006


Epoch 00013: val_categorical_accuracy improved from 0.85199 to 0.85274, saving model to weights1-7.hdf5
Epoch 14/20
11981/11981 [==============================] - 70s 6ms/step - loss: 0.0675 - categorical_accuracy: 0.9800 - val_loss: 0.6339 - val_categorical_accuracy: 0.8542

Epoch 00014: val_categorical_accuracy improved from 0.85274 to 0.85424, saving model to weights1-7.hdf5
Epoch 15/20
11981/11981 [==============================] - 71s 6ms/step - loss: 0.0606 - categorical_accuracy: 0.9831 - val_loss: 0.6908 - val_categorical_accuracy: 0.8535

Epoch 00015: val_categorical_accuracy did not improve from 0.85424
Epoch 16/20
11981/11981 [==============================] - 70s 6ms/step - loss: 0.0486 - categorical_accuracy: 0.9860 - val_loss: 0.7076 - val_categorical_accuracy: 0.8512

Epoch 00016: val_categorical_accuracy did not improve from 0.85424
Epoch 17/20
11981/11981 [==============================] - 70s 6ms/step - loss: 0.0363 - categorical_accuracy: 0.9897 - val_loss: 0.6211 -

...


Epoch 00003: val_categorical_accuracy improved from 0.82733 to 0.86036, saving model to weights5-1.hdf5
Epoch 4/20
11980/11980 [==============================] - 71s 6ms/step - loss: 0.3155 - categorical_accuracy: 0.8929 - val_loss: 0.4392 - val_categorical_accuracy: 0.8596

Epoch 00004: val_categorical_accuracy did not improve from 0.86036
Epoch 5/20
11980/11980 [==============================] - 71s 6ms/step - loss: 0.2575 - categorical_accuracy: 0.9124 - val_loss: 0.4029 - val_categorical_accuracy: 0.8686

Epoch 00005: val_categorical_accuracy improved from 0.86036 to 0.86862, saving model to weights5-1.hdf5
Epoch 6/20
11980/11980 [==============================] - 71s 6ms/step - loss: 0.2058 - categorical_accuracy: 0.9302 - val_loss: 0.4015 - val_categorical_accuracy: 0.8686

Epoch 00006: val_categorical_accuracy improved from 0.86862 to 0.86862, saving model to weights5-1.hdf5
Epoch 7/20
11980/11980 [==============================] - 70s 6ms/step - loss: 0.1724 - categorical_accu

11981/11981 [==============================] - 72s 6ms/step - loss: 0.3106 - categorical_accuracy: 0.8953 - val_loss: 0.3918 - val_categorical_accuracy: 0.8723

Epoch 00004: val_categorical_accuracy improved from 0.86777 to 0.87228, saving model to weights5-4.hdf5
Epoch 5/20
11981/11981 [==============================] - 72s 6ms/step - loss: 0.2668 - categorical_accuracy: 0.9114 - val_loss: 0.3854 - val_categorical_accuracy: 0.8715

Epoch 00005: val_categorical_accuracy did not improve from 0.87228
Epoch 6/20
11981/11981 [==============================] - 72s 6ms/step - loss: 0.2128 - categorical_accuracy: 0.9290 - val_loss: 0.3750 - val_categorical_accuracy: 0.8775

Epoch 00006: val_categorical_accuracy improved from 0.87228 to 0.87754, saving model to weights5-4.hdf5
Epoch 7/20
11981/11981 [==============================] - 73s 6ms/step - loss: 0.1734 - categorical_accuracy: 0.9414 - val_loss: 0.3563 - val_categorical_accuracy: 0.8843

Epoch 00007: val_categorical_accuracy improved f

11981/11981 [==============================] - 71s 6ms/step - loss: 0.2540 - categorical_accuracy: 0.9140 - val_loss: 0.3428 - val_categorical_accuracy: 0.8918

Epoch 00005: val_categorical_accuracy improved from 0.87303 to 0.89181, saving model to weights5-7.hdf5
Epoch 6/20
11981/11981 [==============================] - 71s 6ms/step - loss: 0.2115 - categorical_accuracy: 0.9304 - val_loss: 0.3471 - val_categorical_accuracy: 0.8941

Epoch 00006: val_categorical_accuracy improved from 0.89181 to 0.89406, saving model to weights5-7.hdf5
Epoch 7/20
11981/11981 [==============================] - 71s 6ms/step - loss: 0.1749 - categorical_accuracy: 0.9382 - val_loss: 0.3565 - val_categorical_accuracy: 0.8896

Epoch 00007: val_categorical_accuracy did not improve from 0.89406
Epoch 8/20
11981/11981 [==============================] - 71s 6ms/step - loss: 0.1376 - categorical_accuracy: 0.9532 - val_loss: 0.3630 - val_categorical_accuracy: 0.8956

Epoch 00008: val_categorical_accuracy improved f


Epoch 00016: val_categorical_accuracy did not improve from 0.89932
Epoch 17/20
11981/11981 [==============================] - 71s 6ms/step - loss: 0.0466 - categorical_accuracy: 0.9849 - val_loss: 0.4468 - val_categorical_accuracy: 0.8971

Epoch 00017: val_categorical_accuracy did not improve from 0.89932
Epoch 18/20
11981/11981 [==============================] - 70s 6ms/step - loss: 0.0463 - categorical_accuracy: 0.9837 - val_loss: 0.4862 - val_categorical_accuracy: 0.8926

Epoch 00018: val_categorical_accuracy did not improve from 0.89932
Epoch 19/20
11981/11981 [==============================] - 70s 6ms/step - loss: 0.0419 - categorical_accuracy: 0.9861 - val_loss: 0.4648 - val_categorical_accuracy: 0.8956

Epoch 00019: val_categorical_accuracy did not improve from 0.89932
Epoch 20/20
11981/11981 [==============================] - 70s 6ms/step - loss: 0.0368 - categorical_accuracy: 0.9881 - val_loss: 0.5032 - val_categorical_accuracy: 0.8933

Epoch 00020: val_categorical_accuracy d

,Competência do TCU,Contrato Administrativo,Convênio,Desestatização,Direito Processual,Finanças Públicas,Gestão Administrativa,Licitação,Pessoal,Responsabilidade,accuracy,macro avg,weighted avg
sem pré-treino mean,0.773216,0.754331,0.718178,0.512194,0.881001,0.735516,0.595069,0.906701,0.955620,0.857467,0.866887,0.768929,0.864933
sem pré-treino std,0.037312,0.028111,0.039672,0.152096,0.015771,0.046580,0.109306,0.013776,0.006849,0.019275,0.004813,0.025638,0.005489
NILC fixo mean,0.834367,0.787535,0.736220,0.680333,0.878572,0.710316,0.667027,0.903933,0.956988,0.863169,0.875752,0.801846,0.874368
NILC fixo std,0.026728,0.019094,0.052377,0.110529,0.015674,0.049104,0.084520,0.012355,0.007489,0.012444,0.007750,0.022676,0.007861
NILC variável mean,0.828141,0.814332,0.774899,0.787684,0.904833,0.776430,0.701227,0.923774,0.970883,0.886135,0.897311,0.836834,0.896757
NILC variável std,0.035531,0.040380,0.019612,0.092204,0.018250,0.044454,0.060801,0.012877,0.004837,0.015796,0.006873,0.016430,0.006941
Acordãos fixo mean,0.836235,0.806861,0.784504,0.713898,0.901487,0.763376,0.675837,0.919320,0.967814,0.883952,0.894157,0.825328,0.892949
Acordãos fixo std,0.048380,0.015918,0.038609,0.109981,0.019782,0.053944,0.075166,0.016142,0.007614,0.012567,0.005749,0.019841,0.005914
Acordãos variável mean,0.835062,0.824652,0.788800,0.767185,0.894421,0.784493,0.709311,0.923726,0.969914,0.888006,0.898213,0.838557,0.897405
Acordãos variável std,0.040405,0.020315,0.036093,0.109382,0.014296,0.058275,0.067457,0.016115,0.007775,0.015679,0.006173,0.020968,0.006234


In [15]:
df_medias.T.to_csv('scores_embeddings_gru.csv', encoding = 'Latin1')